In [88]:
# imports

import os
import io
import sys
import json
import requests
import subprocess
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import InferenceClient

In [5]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [6]:
# initialize

openai = OpenAI()
OPENAI_MODEL ="gpt-4o-mini"
QWEN_MODEL="Qwen/Qwen2.5-Coder-32B-Instruct"

In [7]:
client = InferenceClient(
    provider="nebius",
    api_key=os.environ["HF_TOKEN"],
)

In [8]:
system_message = '''
You are a software testing assistant. Your task is to take the code provided by the user and create comprehensive, well-structured unit test cases.

Rules:
1. Use the testing framework most common for the given language (e.g., unittest or pytest for Python, JUnit for Java, etc.).
2. Cover all functions, methods, and classes in the given code.
3. Include:
   - Tests for normal/expected inputs.
   - Tests for edge cases.
   - Tests for invalid inputs and exception handling.
4. Write clear, descriptive test function/method names.
5. Keep tests independent and self-contained.
6. If any dependencies or external resources are used, mock them appropriately.
7. Output only the test code, preserving correct syntax and formatting.
8. Do not modify the original implementation code — only produce the test suite.

'''

In [15]:
def user_prompt_for(code: str) -> str:
    user_prompt = (
        "Please take the following code and write comprehensive, well-structured unit test cases.\n"
        "Rules:\n"
        "1. Use the testing framework most common for the given language (e.g., unittest or pytest for Python).\n"
        "2. Cover all functions, methods, and classes in the code.\n"
        "3. Include tests for normal cases, edge cases, and invalid inputs.\n"
        "4. Ensure tests are independent and self-contained.\n"
        "5. Mock any external dependencies if necessary.\n"
        "6. Output only the complete test code, properly formatted.\n\n"
        "Here is the code:\n\n"
    )
    user_prompt += code
    return user_prompt

In [16]:
def messages_for(code):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(code)}
    ]

In [21]:
# write to a file called optimized.cpp

def write_output(python):
    code = python.replace("```python","").replace("```","")
    with open("unittestcases.py", "w") as f:
        f.write(code)

In [27]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(python+reply)

In [32]:
#Quwen
def optimize_qwen(python):
    stream = client.chat.completions.create(model=QWEN_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(python+reply)

In [28]:
example1 = """
def add(a, b):
    return a + b

def divide(a, b):
    if b == 0:
        raise ValueError("Division by zero is not allowed")
    return a / b

"""

In [29]:
optimize_gpt(example1)

```python
import unittest

class TestMathOperations(unittest.TestCase):
    
    def test_add_with_positive_numbers(self):
        self.assertEqual(add(1, 2), 3)
    
    def test_add_with_negative_numbers(self):
        self.assertEqual(add(-1, -1), -2)
    
    def test_add_with_zero(self):
        self.assertEqual(add(0, 5), 5)
        self.assertEqual(add(5, 0), 5)
        self.assertEqual(add(0, 0), 0)

    def test_add_with_floats(self):
        self.assertEqual(add(1.5, 2.5), 4.0)
    
    def test_divide_with_positive_numbers(self):
        self.assertEqual(divide(10, 2), 5.0)

    def test_divide_with_negative_numbers(self):
        self.assertEqual(divide(-10, -2), 5.0)
    
    def test_divide_with_zero_numerator(self):
        self.assertEqual(divide(0, 5), 0.0)

    def test_divide_with_float(self):
        self.assertEqual(divide(5.0, 2.0), 2.5)

    def test_divide_by_zero_raises_exception(self):
        with self.assertRaises(ValueError) as context:
            divide(1

In [30]:
!python unittestcases.py

.........F.
FAIL: test_divide_with_small_numbers (__main__.TestMathOperations.test_divide_with_small_numbers)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "E:\projects\llm_engineering\week4\Work\unittestcases.py", line 50, in test_divide_with_small_numbers
    self.assertEqual(divide(1e-10, 1e-5), 1e-5)
AssertionError: 9.999999999999999e-06 != 1e-05

----------------------------------------------------------------------
Ran 11 tests in 0.002s

FAILED (failures=1)


In [31]:
optimize_qwen(example1)

```python
import unittest

class TestMathOperations(unittest.TestCase):

    def test_add_with_positive_numbers(self):
        self.assertEqual(add(1, 2), 3)

    def test_add_with_negative_numbers(self):
        self.assertEqual(add(-1, -2), -3)

    def test_add_with_mixed_numbers(self):
        self.assertEqual(add(-1, 2), 1)

    def test_add_with_zero(self):
        self.assertEqual(add(0, 0), 0)
        self.assertEqual(add(0, 5), 5)
        self.assertEqual(add(5, 0), 5)

    def test_divide_with_positive_numbers(self):
        self.assertAlmostEqual(divide(10, 5), 2.0)

    def test_divide_with_negative_numbers(self):
        self.assertAlmostEqual(divide(-10, -5), 2.0)
        self.assertAlmostEqual(divide(-10, 5), -2.0)
        self.assertAlmostEqual(divide(10, -5), -2.0)

    def test_divide_with_mixed_numbers(self):
        self.assertAlmostEqual(divide(0, 5), 0.0)

    def test_divide_by_zero(self):
        with self.assertRaises(ValueError) as context:
            divide(

In [33]:
!python unittestcases.py

.........
----------------------------------------------------------------------
Ran 9 tests in 0.000s

OK


In [34]:
example2 = """
class BankAccount:
    
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self.balance += amount
        return self.balance

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount
        return self.balance

"""

In [35]:
optimize_gpt(example2)

```python
import unittest

class TestBankAccount(unittest.TestCase):

    def setUp(self):
        """Create a bank account instance for testing."""
        self.account = BankAccount("Alice", 100)

    def test_initial_balance(self):
        """Test that the initial balance is set correctly."""
        self.assertEqual(self.account.balance, 100)

    def test_deposit_positive_amount(self):
        """Test depositing a positive amount."""
        new_balance = self.account.deposit(50)
        self.assertEqual(new_balance, 150)

    def test_deposit_zero_amount(self):
        """Test that depositing zero raises a ValueError."""
        with self.assertRaises(ValueError) as context:
            self.account.deposit(0)

        self.assertEqual(str(context.exception), "Deposit amount must be positive")

    def test_deposit_negative_amount(self):
        """Test that depositing a negative amount raises a ValueError."""
        with self.assertRaises(ValueError) as context:
            sel

In [36]:
!python unittestcases.py

.......
----------------------------------------------------------------------
Ran 7 tests in 0.000s

OK


In [37]:
optimize_qwen(example2)

```python
import unittest

class TestBankAccount(unittest.TestCase):

    def setUp(self):
        # Create a new BankAccount instance for each test
        self.account = BankAccount('Test Owner', 100)

    def test_initial_balance(self):
        # Test that initial balance is set correctly
        account = BankAccount('Test Owner')
        self.assertEqual(account.balance, 0)

        # Test that initial balance with non-zero value is set correctly
        account = BankAccount('Test Owner', 100)
        self.assertEqual(account.balance, 100)

    def test_initial_owner(self):
        # Test that owner is set correctly
        self.assertEqual(self.account.owner, 'Test Owner')

    def test_deposit_positive_amount(self):
        # Test depositing a positive amount increases balance correctly
        new_balance = self.account.deposit(50)
        self.assertEqual(new_balance, 150)

    def test_deposit_zero_amount(self):
        # Test depositing zero amount raises ValueError
       

In [38]:
!python unittestcases.py

.........F
FAIL: test_withdraw_zero_amount (__main__.TestBankAccount.test_withdraw_zero_amount)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "E:\projects\llm_engineering\week4\Work\unittestcases.py", line 66, in test_withdraw_zero_amount
    with self.assertRaises(ValueError) as context:
AssertionError: ValueError not raised

----------------------------------------------------------------------
Ran 10 tests in 0.001s

FAILED (failures=1)


In [82]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')
    write_output(python+reply)

In [83]:
def stream_qwen(python):    
    stream = client.chat.completions.create(model=QWEN_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')
    write_output(python+reply)

In [41]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Qwen":
        result = stream_qwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far 

In [63]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Code:", lines=10, value=example1)
        test = gr.Textbox(label="Unit test cases:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Qwen"], label="Select model", value="GPT")
        convert = gr.Button("Generate unit test cases")

    convert.click(optimize, inputs=[python, model], outputs=[test])

ui.launch(inbrowser=True)

In [102]:
def execute_python(_):
    try:
        result = subprocess.run(
            ["python", "unittestcases.py"],  # run the file you saved
            capture_output=True,
            text=True
        )
        return result.stdout + result.stderr
    except Exception as e:
        return str(e)

In [103]:
css = """
.python {background-color: #306998;}
"""

In [104]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Write unite test cases")
    with gr.Row():
        python = gr.Textbox(label="Code:", value=example2, lines=10)
        test = gr.Textbox(label="Unit test cases:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Qwen"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Generate unit test cases")
    with gr.Row():
        python_run = gr.Button("Run")
    with gr.Row():
        python_out = gr.TextArea(label="Test result:", elem_classes=["python"])

    convert.click(optimize, inputs=[python, model], outputs=[test])
    python_run.click(execute_python, inputs=[test], outputs=[python_out])

ui.launch(inbrowser=True)